In [21]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet

import pickle


import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
pd.set_option('display.max_columns', 500)

In [5]:
last_n_dict = pickle.load(open("last_n_dict.p", "rb" ) )

In [6]:
last5 = last_n_dict[5]
last5.head()

,season,player_name,game_date,team,opp,home,wl,mp,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,plus_minus,fscore,ast_pct,blk_pct,dreb_pct,oreb_pct,reb_pct,stl_pct,tov_pct,efg_pct,ts_pct,usg_pct,days_off,game_count,wl_l5,mp_l5,fgm_l5,fga_l5,fg_pct_l5,fg3m_l5,fg3a_l5,fg3_pct_l5,ftm_l5,fta_l5,ft_pct_l5,oreb_l5,dreb_l5,reb_l5,ast_l5,stl_l5,blk_l5,tov_l5,pf_l5,pts_l5,plus_minus_l5,fscore_l5,ast_pct_l5,blk_pct_l5,dreb_pct_l5,oreb_pct_l5,reb_pct_l5,stl_pct_l5,tov_pct_l5,efg_pct_l5,ts_pct_l5,usg_pct_l5,days_off_l5,tm_pace_l5,tm_off_eff_l5,opp_pace_l5,opp_off_eff_l5,opp_def_eff_l5,opp_days_off_l5
0,2000,Peja Stojakovic,2000-10-31,SAC,CHI,0,1,39,10,13,0.769,3,3,1.00,0,1,0.000,1,7,8.0,2,3,0,2,1,23,15,42.50,8.835490,0.000000,23.187803,3.312543,13.250173,3.960990,12.953368,0.884615,0.855655,19.239424,4.0,1,1.0,39.0,10.0,13.0,0.769,3.0,3.0,1.00,0.0,1.0,0.000,1.0,7.0,8.0,2.0,3.0,0.0,2.0,1.0,23.0,15.0,42.50,8.835490,0.000000,23.187803,3.312543,13.250173,3.960990,12.953368,0.884615,0.855655,19.239424,4.0,95.643037,109.291045,95.770863,101.249631,109.514907,4.0
1,2000,Danny Manning,2000-10-31,UTA,LAC,1,1,12,4,7,0.571,0,0,0.00,0,1,0.000,1,2,3.0,0,0,0,2,3,8,-2,10.75,0.000000,0.000000,24.141414,12.070707,18.106061,0.000000,21.186441,0.571429,0.537634,33.986503,4.0,1,1.0,12.0,4.0,7.0,0.571,0.0,0.0,0.00,0.0,1.0,0.000,1.0,2.0,3.0,0.0,0.0,0.0,2.0,3.0,8.0,-2.0,10.75,0.000000,0.000000,24.141414,12.070707,18.106061,0.000000,21.186441,0.571429,0.537634,33.986503,4.0,88.402588,91.218166,98.479220,98.735915,87.539059,4.0
2,2000,David Benoit,2000-10-31,UTA,LAC,1,1,1,0,0,0.000,0,0,0.00,0,1,0.000,0,1,1.0,0,0,0,0,0,0,-2,1.25,0.000000,0.000000,144.848485,0.000000,72.424242,0.000000,0.000000,0.000000,0.000000,19.009400,4.0,1,1.0,1.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,1.0,0.000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,1.25,0.000000,0.000000,144.848485,0.000000,72.424242,0.000000,0.000000,0.000000,0.000000,19.009400,4.0,88.402588,91.218166,98.479220,98.735915,87.539059,4.0
3,2000,DeShawn Stevenson,2000-10-31,UTA,LAC,1,1,7,2,3,0.667,0,0,0.00,1,3,0.333,1,0,1.0,1,1,0,2,0,5,4,8.75,23.271665,0.000000,0.000000,20.692641,10.346320,7.232124,31.645570,0.666667,0.578704,39.006301,4.0,1,1.0,7.0,2.0,3.0,0.667,0.0,0.0,0.00,1.0,3.0,0.333,1.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,5.0,4.0,8.75,23.271665,0.000000,0.000000,20.692641,10.346320,7.232124,31.645570,0.666667,0.578704,39.006301,4.0,88.402588,91.218166,98.479220,98.735915,87.539059,4.0
4,2000,Michael Dickerson,2000-10-31,MEM,OKC,1,1,39,9,26,0.346,2,8,0.25,5,9,0.556,2,3,5.0,3,3,2,2,5,25,8,45.75,13.300221,3.252362,7.269985,4.846657,6.058321,3.845356,6.257822,0.384615,0.417223,35.129209,4.0,1,1.0,39.0,9.0,26.0,0.346,2.0,8.0,0.25,5.0,9.0,0.556,2.0,3.0,5.0,3.0,3.0,2.0,2.0,5.0,25.0,8.0,45.75,13.300221,3.252362,7.269985,4.846657,6.058321,3.845356,6.257822,0.384615,0.417223,35.129209,4.0,88.402588,121.964129,97.413433,106.762971,97.523868,4.0


In [92]:
def year_train_test(data_dict, n=5, year = 2017):
    data = data_dict[n]
    x_train = data.loc[:,'days_off':].loc[data.season != year]
    x_train['home'] = data.home.loc[data.season != year]
    y_train = data.loc[data.season != year].fscore

    x_test = data.loc[:,'days_off':].loc[data.season == year]
    x_test['home'] = data.home.loc[data.season == year]
    y_test = data.loc[data.season == year].fscore
    
    return x_train, x_test, y_train, y_test

def szn_train_test(data_dict, n = 5, year = 2017):
    data = data_dict[n]
    X = data.loc[:,'days_off':].loc[data.season == year]
    X['home'] = data.home.loc[data.season == year]
    y = data.loc[data.season == year].fscore
    return train_test_split(X,y, test_size = .2, random_state=1)

def plyr_train_test(data_dict, player, n = 5):
    data = data_dict[n]
    player_df = data.loc[data.player_name == player]
    x_train = player_df.loc[:,'days_off':][:round((.9*len(player_df)))]
    x_train['home'] = data.home.loc[data.player_name == player]
    y_train = player_df.fscore[:round((.9*len(player_df)))]

    x_test = player_df.loc[:,'days_off':][round((.9*len(player_df))):]
    x_test['home'] = data.home.loc[data.player_name == player]
    y_test = player_df.fscore[round((.9*len(player_df))):]
    
    return x_train, x_test, y_train, y_test

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())



In [84]:
X_train,X_test,y_train,y_test = plyr_train_test(last_n_dict, 'James Harden')


In [86]:
from sklearn.feature_selection import SelectKBest, f_regression
X_scaled = StandardScaler().fit_transform(X_train)
kbest = SelectKBest(f_regression, k=10)

kbest.fit(X_scaled,y_train)
for label in X_train.columns[kbest.get_support()]:
    print(label)


fga_l5
fg3a_l5
ftm_l5
fta_l5
ast_l5
tov_l5
pts_l5
fscore_l5
ast_pct_l5
usg_pct_l5


In [87]:
fts_picked = ['mp_l5', 'pts_l5','ast_l5','reb_l5','stl_l5','blk_l5','tov_l5','fg3m_l5','opp_def_eff_l5','tm_pace_l5', 'home']

fts_kbest = X_train.columns[kbest.get_support()]
Xp_train, Xp_test = X_train[fts_picked], X_test[fts_picked]
Xk_train, Xk_test = X_train[fts_kbest], X_test[fts_kbest]

In [88]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_scaled = StandardScaler().fit_transform(Xk_train)
lin_reg = LinearRegression()
lin_reg.fit(X_scaled, y_train)
y_pred = lin_reg.predict(X_scaled)
lin_mse = mean_squared_error(y_train, y_pred)
lin_rmse = np.sqrt(lin_mse)
lin_rmse


12.179944315156789

In [89]:
lin_scores = cross_val_score(lin_reg, X_scaled, y_train,
                             scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [ 9.88150551  8.87240552  7.77631159 12.44471634 13.04929248 14.58597731
 13.67947417 13.86267047 12.73213558 15.16238195]
Mean: 12.204687091876794
Standard deviation: 2.3770436406615465


In [135]:
def avg_rmse(X,y, model, cv=5):
    scores = cross_val_score(model, X, y,
                             scoring="neg_mean_squared_error", cv=cv)
    rmse_scores = np.sqrt(-scores)
    return rmse_scores.mean()

def scaled_ridge(alpha = 1):
    sc_ridge = Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", Ridge(alpha = alpha)),
        ]) 
    return sc_ridge
    
def scaled_ols(alpha = 1):
    sc_ols = Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", LinearRegression()),
        ]) 
    return sc_ols

In [129]:
avg_rmse(X_train, y_train, scaled_ridge())

9.498254259914033

In [130]:
avg_rmse(X_train, y_train, scaled_ols())

9.499403619614046

In [120]:
X_train,X_test,y_train,y_test = plyr_train_test(last_n_dict, player = 'James Harden', n=10)
X_train.tail()

,days_off,game_count,wl_l10,mp_l10,fgm_l10,fga_l10,fg_pct_l10,fg3m_l10,fg3a_l10,fg3_pct_l10,ftm_l10,fta_l10,ft_pct_l10,oreb_l10,dreb_l10,reb_l10,ast_l10,stl_l10,blk_l10,tov_l10,pf_l10,pts_l10,plus_minus_l10,fscore_l10,ast_pct_l10,blk_pct_l10,dreb_pct_l10,oreb_pct_l10,reb_pct_l10,stl_pct_l10,tov_pct_l10,efg_pct_l10,ts_pct_l10,usg_pct_l10,days_off_l10,tm_pace_l10,tm_off_eff_l10,opp_pace_l10,opp_off_eff_l10,opp_def_eff_l10,opp_days_off_l10,home
407855,1.0,80,0.6,37.4,9.1,22.4,0.4041,3.5,11.2,0.3046,9.2,10.6,0.8688,1.3,7.4,8.7,11.2,1.2,0.6,5.5,2.9,30.9,2.8,63.425,50.617632,1.539386,21.630864,3.830981,12.730923,1.464179,16.853310,0.482305,0.569846,36.690188,2.2,103.023888,113.009767,96.159307,117.512413,108.063909,2.1,0
408359,2.0,81,0.5,36.1,8.2,21.0,0.3785,3.3,11.0,0.2838,8.6,10.3,0.8449,1.4,7.0,8.4,10.8,1.1,0.6,5.2,3.1,28.3,-0.9,59.250,49.232415,1.539386,21.831534,4.316753,13.074144,1.409741,17.254467,0.455735,0.550403,35.851614,2.2,102.346341,110.682278,94.701358,114.177128,117.532271,1.8,1
408413,4.0,1,0.5,35.6,7.7,19.4,0.3952,3.4,10.9,0.2974,8.3,10.0,0.8381,1.4,7.3,8.7,10.9,1.1,0.7,5.0,3.0,27.1,0.8,59.225,49.605576,1.875275,22.411474,4.266905,13.339189,1.508470,17.697760,0.484780,0.574498,33.971159,2.2,102.230768,111.617926,97.573787,117.950337,105.917641,1.9,0
408489,1.0,2,0.5,35.4,7.5,19.3,0.3887,3.3,10.6,0.3001,7.7,9.3,0.8313,1.4,7.2,8.6,10.3,1.0,0.6,5.0,3.0,26.0,-0.3,56.650,46.491457,1.641357,22.670834,4.361596,13.516215,1.396872,17.836241,0.476537,0.563141,33.754672,2.2,101.932351,110.133965,95.600007,107.273784,112.235020,1.9,0
408941,3.0,3,0.5,35.6,7.5,19.9,0.3735,3.5,11.2,0.3037,8.0,9.6,0.8388,1.6,6.8,8.4,10.0,1.2,0.6,4.7,2.6,26.5,-1.3,56.950,47.389177,1.641357,21.106063,4.935759,13.020911,1.665603,16.338136,0.464156,0.554244,34.046526,2.1,101.524803,110.692594,93.359684,104.730470,112.638276,2.0,1


In [131]:
for key in sorted(last_n_dict.keys()):
    X_train,X_test,y_train,y_test = plyr_train_test(last_n_dict, player='James Harden', n=key)
    rmse = avg_rmse(X_train,y_train,scaled_ridge(),10)
    print('last',key)
    print("Mean: {}".format(rmse),'\n')

last 1
Mean: 13.864058501558569 

last 3
Mean: 12.57177796202029 

last 5
Mean: 12.635259928802828 

last 7
Mean: 12.232385560796066 

last 10
Mean: 12.71415536832519 

last 15
Mean: 12.636801126867383 

last 20
Mean: 12.7531671640268 



In [132]:
for key in sorted(last_n_dict.keys()):
    X_train,X_test,y_train,y_test = szn_train_test(last_n_dict, n=key)
    rmse = avg_rmse(X_train,y_train,scaled_ridge())
    print('last',key)
    print("Mean: {}".format(rmse),'\n')

last 1
Mean: 10.426360039490955 

last 3
Mean: 9.708996287994168 

last 5
Mean: 9.555885164682442 

last 7
Mean: 9.49665810590852 

last 10
Mean: 9.46715351050826 

last 15
Mean: 9.480490973369353 

last 20
Mean: 9.498254259914033 



In [206]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_scaled, y)
y_pred = tree_reg.predict(X_scaled)
tree_mse = mean_squared_error(y, y_pred)
tree_mse = np.sqrt(tree_mse)

scores = cross_val_score(tree_reg, X_scaled, y,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [13.97640612 13.65213664 13.22381261 13.76487604 13.41301207 13.74158748
 14.03585916 13.75202097 13.90963316 13.70897385]
Mean: 13.717831810880995
Standard deviation: 0.2345422301317782


In [145]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
def scaled_tree():
    sc_tree = Pipeline([
        ("scaler", StandardScaler()),
        ("tree", DecisionTreeRegressor()),
        ]) 
    return sc_tree

def scaled_forest():
    sc_forest = Pipeline([
        ("scaler", StandardScaler()),
        ("tree", RandomForestRegressor()),
        ]) 
    return sc_forest

In [146]:
for key in sorted(last_n_dict.keys()):
    X_train,X_test,y_train,y_test = plyr_train_test(last_n_dict, n=key, player ='James Harden')
    rmse = avg_rmse(X_train,y_train,scaled_tree())
    print('last',key)
    print("Mean: {}".format(rmse),'\n')

last 1
Mean: 19.392899095633332 

last 3
Mean: 19.67674567659305 

last 5
Mean: 20.43842973190835 

last 7
Mean: 20.167115661381864 

last 10
Mean: 21.86571561793449 

last 15
Mean: 18.647902681122247 

last 20
Mean: 18.700200257905358 



In [144]:
from sklearn.ensemble import RandomForestRegressor
X_scaled = StandardScaler().fit_transform(X_train)
forest_reg = RandomForestRegressor()

scores = cross_val_score(forest_reg, X_scaled, y_train,
                         scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)

display_scores(forest_rmse_scores)

Scores: [ 9.88950155  9.68532196  9.94665553 10.17160218  9.78920498]
Mean: 9.896457239713097
Standard deviation: 0.16392324112886925


In [147]:
for key in sorted(last_n_dict.keys()):
    X_train,X_test,y_train,y_test = plyr_train_test(last_n_dict, n=key, player ='James Harden')
    rmse = avg_rmse(X_train,y_train,scaled_forest())
    print('last',key)
    print("Mean: {}".format(rmse),'\n')

last 1
Mean: 15.870027861121022 

last 3
Mean: 15.094678706182298 

last 5
Mean: 15.049517496113713 

last 7
Mean: 14.116438487836257 

last 10
Mean: 15.485329393177574 

last 15
Mean: 14.91177466912156 

last 20
Mean: 14.770773721606512 



In [213]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error')

grid_search.fit(Xk_scaled, y)
grid_search.best_params_

{'max_features': 2, 'n_estimators': 30}

In [216]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [217]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([0.08833318, 0.05150846, 0.09634537, 0.02473211, 0.02643539,
       0.03573149, 0.05136041, 0.03114714, 0.02193507, 0.05147601,
       0.03875832, 0.04345901, 0.03396039, 0.01883354, 0.03853961,
       0.0281286 , 0.10777743, 0.1271779 , 0.03942741, 0.04493315])

In [218]:
sorted(zip(feature_importances, Xk.columns), reverse=True)

[(0.12717790345210028, 'fscore_l5'),
 (0.10777742859267825, 'pts_l5'),
 (0.0963453707421805, 'fga_l5'),
 (0.08833317773232957, 'mp_l5'),
 (0.05150846241107802, 'fgm_l5'),
 (0.05147601453873185, 'dreb_l5'),
 (0.0513604053568286, 'fta_l5'),
 (0.04493315083028555, 'usg_pct_l5'),
 (0.043459011839729306, 'ast_l5'),
 (0.03942740613088648, 'ast_pct_l5'),
 (0.03875832120788174, 'reb_l5'),
 (0.03853961335229258, 'tov_l5'),
 (0.03573149405590996, 'ftm_l5'),
 (0.033960390001929436, 'stl_l5'),
 (0.031147135898747607, 'ft_pct_l5'),
 (0.028128595383345046, 'pf_l5'),
 (0.026435393416842517, 'fg3a_l5'),
 (0.0247321092274293, 'fg3m_l5'),
 (0.02193507089978283, 'oreb_l5'),
 (0.018833544929010605, 'blk_l5')]

In [219]:
final_model = grid_search.best_estimator_

X_test_prepared = StandardScaler().fit_transform(Xk_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(ytest, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

9.692609591698837

In [ ]:
def lin_reg(data, reg_type, func, year=None, player=None):
    if player != None:
        X_train,X_test,y_train,y_test = func(data, player)
    elif year != None:
        X_train,X_test,y_train,y_test = func(data, year)
    else:
        print('Enter year or player')
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("reg", reg_type()),
        ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_train)
    lin_mse = mean_squared_error(y_train, y_pred)
    lin_rmse = np.sqrt(lin_mse)
    lin_scores = cross_val_score(pipe, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=10)

    lin_rmse_scores = np.sqrt(-lin_scores)
    return lin_rmse_scores.mean()